In [13]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv('../data/public_comments.csv')
df_133 = pd.read_csv('../data/public_comments_133.csv')

In [7]:
df_133

,id,type,attributes_document_type,attributes_last_modified_date,attributes_highlighted_content,attributes_withdrawn,attributes_agency_id,attributes_title,attributes_object_id,attributes_posted_date,...,clean_comment,sentiment_TEXTBLOB,anger,disgust,fear,joy,neutral,sadness,surprise,job
0,VA-2020-VHA-0024-9010,comments,Public Submission,2020-12-15T05:40:14Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648499d50c,2020-12-15T05:00:00Z,...,As a Certified Registered Nurse Anesthetist CR...,0.183001,0.035966,0.042263,0.030267,0.106850,0.763856,0.016770,0.004027,Certified Registered Nurse Anesthetist Crna
1,VA-2020-VHA-0024-7682,comments,Public Submission,2020-12-08T13:09:48Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648498c1e0,2020-12-08T05:00:00Z,...,"Our nations Veterans deserve safe, high quali...",0.161527,0.081029,0.018753,0.604690,0.063061,0.202664,0.023710,0.006093,Physician Anesthesiologist
2,VA-2020-VHA-0024-4675,comments,Public Submission,2020-11-24T15:28:02Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,0900006484971563,2020-11-24T05:00:00Z,...,As a Certified Registered Nurse Anesthetist CR...,0.164127,0.038222,0.023488,0.057231,0.111218,0.756551,0.008178,0.005112,Certified Registered Nurse Anesthetist Crna
3,VA-2020-VHA-0024-6159,comments,Public Submission,2020-12-04T16:49:12Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648498635f,2020-12-04T05:00:00Z,...,"Our nations Veterans deserve safe, high quali...",0.161527,0.081982,0.018169,0.605877,0.066199,0.197936,0.023809,0.006029,Physician Anesthesiologist
4,VA-2020-VHA-0024-9495,comments,Public Submission,2020-12-21T12:43:28Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849ab9a8,2020-12-21T05:00:00Z,...,As a Certified Registered Nurse Anesthetist CR...,0.164127,0.038222,0.023488,0.057231,0.111218,0.756551,0.008178,0.005112,Certified Registered Nurse Anesthetist Crna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,VA-2020-VHA-0024-0464,comments,Public Submission,2020-11-14T16:52:03Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495e9b6,2020-11-14T05:00:00Z,...,"As a concerned citizen, Im writing today in su...",0.164127,0.163344,0.021739,0.504148,0.022573,0.263576,0.021889,0.002730,Concerned Citizen
129,VA-2020-VHA-0024-4537,comments,Public Submission,2020-11-24T15:27:00Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849704e0,2020-11-24T05:00:00Z,...,"As a physician anesthesiologist, I believe th...",0.154154,0.084285,0.013342,0.800774,0.014158,0.067417,0.017181,0.002842,Physician Anesthesiologist
130,VA-2020-VHA-0024-4381,comments,Public Submission,2020-11-24T15:25:47Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648496fb94,2020-11-24T05:00:00Z,...,As a Certified Registered Nurse Anesthetist CR...,0.164127,0.038222,0.023488,0.057231,0.111218,0.756551,0.008178,0.005112,Certified Registered Nurse Anesthetist Crna
131,VA-2020-VHA-0024-5175,comments,Public Submission,2020-11-27T13:13:38Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,0900006484976467,2020-11-27T05:00:00Z,...,Our nations Veterans have served and sacrific...,0.092715,0.054115,0.009591,0.804570,0.016333,0.070199,0.035748,0.009444,NaN


In [4]:
df.head()

,id,type,attributes_document_type,attributes_last_modified_date,attributes_highlighted_content,attributes_withdrawn,attributes_agency_id,attributes_title,attributes_object_id,attributes_posted_date,links_self,comment
0,VA-2020-VHA-0024-0004,comments,Public Submission,2020-11-12T19:26:52Z,NaN,False,VA,"Comment on AQ94-Interim Final Rule-Batley, Tyler",090000648495a6f1,2020-11-12T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,&quot; Directive 1899 memorializes U.S. Depart...
1,VA-2020-VHA-0024-0005,comments,Public Submission,2020-11-12T19:34:12Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Anonymous,090000648495accf,2020-11-12T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"As a family physician, I oppose autonomous pra..."
2,VA-2020-VHA-0024-0006,comments,Public Submission,2020-11-13T13:28:22Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495a87a,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,Our nations Veterans have served and sacrifice...
3,VA-2020-VHA-0024-0007,comments,Public Submission,2020-11-13T13:28:22Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495b402,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,This proposed regulation is a danger to health...
4,VA-2020-VHA-0024-0008,comments,Public Submission,2020-11-13T13:28:23Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495b5f8,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,This seems like just common sense. <br/>The VA...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13277 entries, 0 to 13276
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              13277 non-null  object 
 1   type                            13277 non-null  object 
 2   attributes_document_type        13277 non-null  object 
 3   attributes_last_modified_date   13277 non-null  object 
 4   attributes_highlighted_content  0 non-null      float64
 5   attributes_withdrawn            13277 non-null  bool   
 6   attributes_agency_id            13277 non-null  object 
 7   attributes_title                13277 non-null  object 
 8   attributes_object_id            13277 non-null  object 
 9   attributes_posted_date          13277 non-null  object 
 10  links_self                      13277 non-null  object 
 11  comment                         13277 non-null  object 
dtypes: bool(1), float64(1), object(1

In [16]:
def nlp_clean(str_to_clean: str) -> str:
    """
    cleans strings for use with nlp services

    """
    clean_str = re.sub(
        r"<[^>]+>",
        " ",
        str_to_clean,
    )
    clean_str = re.sub(
        r"&quot;",
        "",
        clean_str,
    )
    clean_str = re.sub(
        r"&#39;",
        "",
        clean_str,
    )
    clean_str = re.sub(
        r"-",
        " ",
        clean_str,
    )
    clean_str = re.sub(
        r"[^a-zA-Z0-9 \.!,]",
        "",
        clean_str,
    )
    string_encode = clean_str.encode("ascii", "ignore")
    string_decode = string_encode.decode()

    return string_decode


def get_cosine_sim(df_full):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df_full["clean_comment"])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim



def get_similar_comments(
    index: int, df: pd.DataFrame, cosine_sim, threshold: float
) -> pd.DataFrame:
    """
    Returns a list of comments that are similiar to the comment passed in.
    """
    new_df = df
    new_df["Similar_Score"] = cosine_sim[index].tolist()
    new_df = new_df.loc[
        new_df["Similar_Score"] > threshold,
    ]

    return new_df

In [8]:
df["clean_comment"] = df["comment"].apply(nlp_clean)

In [9]:
df

,id,type,attributes_document_type,attributes_last_modified_date,attributes_highlighted_content,attributes_withdrawn,attributes_agency_id,attributes_title,attributes_object_id,attributes_posted_date,links_self,comment,clean_comment
0,VA-2020-VHA-0024-0004,comments,Public Submission,2020-11-12T19:26:52Z,NaN,False,VA,"Comment on AQ94-Interim Final Rule-Batley, Tyler",090000648495a6f1,2020-11-12T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,&quot; Directive 1899 memorializes U.S. Depart...,Directive 1899 memorializes U.S. Department o...
1,VA-2020-VHA-0024-0005,comments,Public Submission,2020-11-12T19:34:12Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Anonymous,090000648495accf,2020-11-12T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"As a family physician, I oppose autonomous pra...","As a family physician, I oppose autonomous pra..."
2,VA-2020-VHA-0024-0006,comments,Public Submission,2020-11-13T13:28:22Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495a87a,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,Our nations Veterans have served and sacrifice...,Our nations Veterans have served and sacrifice...
3,VA-2020-VHA-0024-0007,comments,Public Submission,2020-11-13T13:28:22Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495b402,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,This proposed regulation is a danger to health...,This proposed regulation is a danger to health...
4,VA-2020-VHA-0024-0008,comments,Public Submission,2020-11-13T13:28:23Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,090000648495b5f8,2020-11-13T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,This seems like just common sense. <br/>The VA...,This seems like just common sense. The VA sho...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13272,VA-2020-VHA-0024-13329,comments,Public Submission,2021-01-12T04:56:00Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849eee1a,2021-01-11T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"<br/>Veterans fight for our country, we should...","Veterans fight for our country, we should fig..."
13273,VA-2020-VHA-0024-13330,comments,Public Submission,2021-01-12T04:56:01Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849eee1b,2021-01-11T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"As a concerned citizen, Im writing today in su...","As a concerned citizen, Im writing today in su..."
13274,VA-2020-VHA-0024-13330,comments,Public Submission,2021-01-12T04:56:01Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849eee1b,2021-01-11T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"As a concerned citizen, Im writing today in su...","As a concerned citizen, Im writing today in su..."
13275,VA-2020-VHA-0024-13330,comments,Public Submission,2021-01-12T04:56:01Z,NaN,False,VA,Comment on AQ94-Interim Final Rule-Comment Sub...,09000064849eee1b,2021-01-11T05:00:00Z,https://api.regulations.gov/v4/comments/VA-202...,"As a concerned citizen, Im writing today in su...","As a concerned citizen, Im writing today in su..."


In [15]:
vectorizer = TfidfVectorizer()

In [17]:
tfidf_matrix = vectorizer.fit_transform(df["clean_comment"])

In [18]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim

In [ ]:
cosine_sim = get_cosine_sim(df)

In [ ]:
number = 5
limit = 0.95
sim_comments = get_similar_comments(
    index=number, df=df, cosine_sim=cosine_sim, threshold=limit,
)